In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,TensorDataset

The Bernoulli Gamma Loss Function

In [40]:
class BGLoss(nn.Module):
    
    def __init__(self):
        super(BGLoss,self).__init__()

    def forward(self, p_pred, log_alpha_pred, log_beta_pred,y_train):

            '''Computes the BG loss for the given batch for the purposes of downscaling precipitation
            Parameters:
            p_pred: Predicted probability of precipitation
            log_alpha_pred: Predicted parameter for Gamma Distribution
            log_beta_pred: Predicted parameter for Gamma Distribution
            y_true: True Precip value
            
            The function returns the total loss combining the Bernoulli and Gamma losses'''

            #Creating a binary mask for precipitation : 1 if precip exceeds 0, 0 otherwise , denoting occurence of precipitation

    occurence_mask=(y_train>0).float()


    bernoulli_loss=  occurence_mask* torch.log(p_pred+1e-10) + (1-occurence_mask)*torch.log(1-p_pred+1e-10)
    bernoulli_loss=torch.mean(bernoulli_loss)


        #Computing the Gamma loss

    if occurence_mask.sum()>0:
            alpha= torch.exp(log_alpha_pred)
            beta=torch.exp(log_beta_pred)

                #Negative log likelihood
            gamma_loss= alpha*torch.log(beta+1e-10) -torch.lgamma(alpha) + (alpha-1)*torch.log(y_train+1e-10) - beta*y_train
            gamma_loss=-gamma_loss
            gamma_loss=gamma_loss*occurence_mask.sum()/occurence_mask.sum() #Normalising over the raining points

    else:
            gamma_loss=torch.tensor(0.0,device=y_train.device)

total_loss=bernoulli_loss+gamma_loss
return total_loss  


RuntimeError: The size of tensor a (32) must match the size of tensor b (100) at non-singleton dimension 1

CNN Model for Downscaling Precipitation

In [5]:
# CNN Model for Downscaling Precipitation
class PrecipDownscalingCNN(nn.Module):
    def __init__(self, input_shape, output_size):
        super(PrecipDownscalingCNN, self).__init__()
        
        # Convolutional layers with SAME padding
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=25, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=25, out_channels=1, kernel_size=3, padding=1)
        
        # Flatten the feature maps
        self.flatten = nn.Flatten()
        
        # Fully connected layers for precipitation prediction
        flattened_size = output_size * output_size  # Assuming input has spatial dimensions preserved
        self.fc_p = nn.Linear(flattened_size, output_size)  # Precipitation occurrence (sigmoid)
        self.fc_log_alpha = nn.Linear(flattened_size, output_size)  # Gamma distribution parameter
        self.fc_log_beta = nn.Linear(flattened_size, output_size)  # Gamma distribution parameter
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)
        
        # Three outputs
        p_pred = torch.sigmoid(self.fc_p(x))  # Probability of precipitation
        log_alpha = self.fc_log_alpha(x)  # Gamma parameter
        log_beta = self.fc_log_beta(x)  # Gamma parameter
        
        return torch.cat((p_pred, log_alpha, log_beta), dim=1)


Data Processing

In [6]:
def Scale_Data(x):
    '''Standadizes Input Precip Data: mean 0 unit variance'''
    mean= x.mean()
    std=x.std()
    return (x-mean)/(std +1e-10), mean, std

def binary_precipitation(y):
    '''0 if it didnt rain, 1 if it did'''

    return(y>0.99).float()



Model hyperparameters

In [39]:
input_shape = (1, 100, 100)  # Assuming 32x32 input...will be changed later
output_size=100 # Assuming 32x32 output...will be changed later
lr=0.0001
batch_size=100
epochs=10000


In [43]:
#Example data for training
x_train=torch.randn(100,1,32,32)
y_train=torch.randn(100,1,32,32)
y_train=binary_precipitation(y_train)

In [44]:
model_precip_V0=PrecipDownscalingCNN(input_shape,output_size)
loss_fn=BGLoss() #Bernoulli Gamma Loss, defined and created before
optimizer=optim.Adam(model_precip_V0.parameters(),lr=lr)

#Training loop
for epoch in range(epochs):
    model_precip_V0.train()
    optimizer.zero_grad()
    pred=model_precip_V0(x_train)

    #Predicted values
    p_pred=pred[:, :y_train.size(1)]
    log_alpha_pred= pred[:, y_train.size(1):2*y_train.size(1)]
    log_beta_pred= pred[:,2*y_train.size(1):]

    loss_fn=BGLoss()
    #Computing the loss
    loss= loss_fn(p_pred, log_alpha_pred, log_beta_pred, y_train)
    loss.backward()

    optimizer.step()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1024 and 10000x100)

In [24]:
#Model prediction
def predict_precipitation(model,x):
    '''Predicts precipitation for a given set of inputs'''

    with torch.no_grad():
        model.eval()
        pred=model(x)
        p_pred(pred[:, :y_train.size(1)])
        log_alpha_pred(pred[:, y_train.size(1):2*y_train.size(1)])
        log_beta_pred(pred[:,2*y_train.size(1):]) 

        return p_pred,log_alpha_pred,log_beta_pred          

In [27]:
#Saving predictions
def save_predictions(model,x,y):
    '''Saves the predictions for a given set of inputs'''

    ds=xr.Dataset({'precip':(['time','lat','lon'],y),
                   'p_pred':(['time','lat','lon'],p_pred)})

In [41]:
#Check the shape of the tensors 

In [42]:
print("Shape of pred:", pred.shape)
print("Shape of p_pred:", p_pred.shape)
print("Shape of log_alpha_pred:", log_alpha_pred.shape)
print("Shape of log_beta_pred:", log_beta_pred.shape)
print("Shape of y_train:", y_train.shape)

Shape of pred: torch.Size([100, 96])
Shape of p_pred: torch.Size([100, 32])
Shape of log_alpha_pred: torch.Size([100, 32])
Shape of log_beta_pred: torch.Size([100, 32])
Shape of y_train: torch.Size([100, 32, 32])
